In [1]:
import os

In [2]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [46]:
from pathlib import Path
CONFIG_FILE_PATH = Path("/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/config/config.yaml")
PARAMS_FILE_PATH = Path("/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/params.yaml")

In [47]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_num_train_epochs: int
    params_per_device_train_batch_size: int
    params_per_device_eval_batch_size: int
    params_weight_decay: float
    params_warmup_steps: int
    params_max_steps: int
    params_save_steps: int
    params_logging_steps: int
    

In [48]:
from airlinesSentiment.constants import *
from airlinesSentiment.utils.common import read_yaml, create_directories

In [57]:


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_num_train_epochs=self.params.num_train_epochs,
            params_per_device_train_batch_size=self.params.per_device_train_batch_size,
            params_per_device_eval_batch_size=self.params.per_device_eval_batch_size,
            params_weight_decay=self.params.weight_decay,
            params_warmup_steps=self.params.warmup_steps,
            params_max_steps=self.params.max_steps,
            params_save_steps=self.params.save_steps,
            params_logging_steps=self.params.logging_steps
            
        )

        return prepare_base_model_config

In [61]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        model = AutoModelForSequenceClassification.from_pretrained(
            "distilbert-base-uncased", num_labels=3
        )

        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

        self.save_model(model, tokenizer)

    
    def save_model(self, model, tokenizer):
        save_path = self.config.base_model_path
        save_path.parent.mkdir(parents=True, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)

        print(f"Base Model saved to {save_path}")
        print(f"Tokenizer saved to {save_path}")

        

In [62]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [63]:
if __name__ == "__main__":

    config_manager = ConfigurationManager()

    prepare_base_model_config = config_manager.get_prepare_base_model_config()

    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

    # prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

    prepare_base_model.get_base_model()




[2025-08-07 13:18:08,202: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-07 13:18:08,209: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-07 13:18:08,211: INFO: common: created directory at: artifacts]
[2025-08-07 13:18:08,211: INFO: common: created directory at: artifacts/prepare_model]


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base Model saved to artifacts/prepare_model
Tokenizer saved to artifacts/prepare_model
